# CA2 - Statistical Insights on Agriculture in Ireland and EU

## Overview

In this notebook, the following agriculture datasets were analysed, focusing on Ireland and other similar countries in the EU:

- Organic Farming Growth
- Agriculture Import/Export Trade
- Food Price Inflation

The datasets were studied and the following statistical techniques were applied:

- Confidence interval

- Hypoteshis tests:
    - Shapiro
    - Levene
    - Non-parametric tests:
        - Kruskal
        - Mannwhitneyu
        - Wilcoxon
    - Parametric test:
        - Anova One-Way

This notebook is divided into two parts (i) Confidence Interval and (ii) Hypothesis tests. 

In the first part, the Organic Farming dataset were used to find a confidence interval for the average area under organic farming in the EU.

In the second part, the Import/Export and Food Price Inflation datasets were checked against some inferential statistics tests.

## Auxiliary Functions

In this section, the auxiliary functions used in this notebook were implemented.

In [1]:
# adding modules folder to the path
import sys
sys.path.append('./modules')

# importing constants defined for this project
from constants import *

# importing modules specific to this notebook
# uncomment to install any missing lib
#!pip install altair
#!pip install pingouin
from IPython.display import Image
from pingouin import kruskal
from math import sqrt
from scipy.stats import wilcoxon
from scipy.stats import levene
from scipy import stats
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

def is_dist_normal(data_col):
    """
    This function applies the Shapiro test to decide
    if the data is from a normal distribution or not.

    Parameters:
        data_col (data series): DataSeries
    Return:
        (bool): True if data is normal, false otherwise.
    """
    shapiro_test = stats.shapiro(data_col)
    if shapiro_test.pvalue > 0.05:
        return True
    return False
    
def plot_col_graph(data_col, title=''):
    """
    This function plots a graph for the column specified
    in the input.

    Parameters:
        data_col (data series): DataSeries
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    stats.probplot(data_col, plot=ax)
    ax.set_title(title)

def get_non_empty_rows(col_data):
    """
    This function removes empty rows from data frame series (column).

    Parameters:
        col_data (pd.Series): DataFrame series
    Returns:
        Non empty rows
    """
    null_filter = col_data.notnull()
    return col_data[null_filter]

def conf_int_ts_student(mean, std, n, alpha=.97):
    """
    This function calculates the confidence interval for the given paramters.
    
    Parameters:
        mean (num): Average of the sample
        std  (num): Standard deviation of the sample
        n    (int): Size of sample
    Returns:
        (num, num): (lower limit, upper limit)
    """
    t = abs(stats.t.ppf((1 - alpha)/2, n - 1))
    interval = t * std/sqrt(n - 1)
    return mean - interval, mean + interval

def get_stats_array(stats_dict, val='mean'):
    """
    This function extracts the stats from the dictionary into a numpy array.
    
    Parameters:
        stats_dict (dict): Dictionary with custom statistics
        val         (str): Measurement to return (must be one of [mean, std, count, ll, ul])
    Returns:
        np array with values as per parameter or empty if input is invalid.
    """
    if val not in ['mean', 'std', 'count', 'll', 'ul']:
        return np.array([])
    return np.array(list([stats_dict[year][val] for year in stats_dict]))

## Confidence Interval

For this study, the organic farming data from Eurostat portal were used.

In this analysis the sample data on the growth of organic farming in several European countries were used to obtain a confidence interval of the same growth in Europe (population) over the years.

The steps of this analysis were detailed in the section below.

### Organic Farming

Eurostat provides yearly data on the percent of UAA (utilized agricultural area) under organic farming per country in Europe.

Problem: What is the average percent of land of the total UAA that is occupied by organic farming in Europe as a whole? How does it compare to Ireland?

- Available data:

    * Percent of total UAA under organic farming by year by country in Europe.

- Approach:

    * Use countries data (samples) to find a confidence interval for the total UAA under organic farming in Europe (population) per year. 

    * Compare the growth rate of organic farming in Europe with Ireland.

### Data Preparation and Visualization

The steps performed to prepare this dataset were implemented in a separate Jupyter notebook called DataPrepVis under this directory.

Thus, the processed datasets were used directly in this notebook.

In [ ]:
# getting organic farm dataset processed dataset
dataset = pd.read_csv(join(DATA_PREP_FOLDER, ORG_FARM_DATASET), encoding='utf-8')
dataset.head()

In [ ]:
# getting irish organic farm dataset processed dataset
dataset_ire = pd.read_csv(join(DATA_PREP_FOLDER, ORG_FARM_DATASET_IRE), encoding='utf-8')
dataset_ire.head()

### Analysis

For each year (column) it was calculated the standard deviation and average of the percent of land under organic farming in all European countries in the dataset.

This way, it was possible to calculate the confidence interval of the population (Europe) per year.

For each year, it was stored the standard deviation, mean, lower and upper limit obtained from the confidence interval calculation.

In [ ]:
# calculate descriptive stats (standard deviation and average) of the sample data - without nulls
stats_dict = {}
for c in dataset.keys():
    dataset_c = get_non_empty_rows(dataset[c])
    desc_stats = dataset_c.describe()
    stats_dict[c] = {'std': desc_stats.loc['std'],
                     'mean': desc_stats.loc['mean'],
                     'count': desc_stats.loc['count']}
    stats_dict[c]['ll'], stats_dict[c]['ul'] = conf_int_ts_student(stats_dict[c]['mean'], stats_dict[c]['std'],  stats_dict[c]['count'])
stats_dict

### Results

In this section, the results of the confidence interval obtained in the previous step were plotted into a graph.

This way, it's possible to observe how the size of organic farming has grown in Ireland as compared to Europe, using the confidence interval calculated from the sample data.

In [ ]:
# get x data
x = np.array(list(stats_dict.keys()))

# get y data
y = get_stats_array(stats_dict, 'mean')

# get ll data
ll = get_stats_array(stats_dict, 'll')

# get ul data
ul = get_stats_array(stats_dict, 'ul')

# get ireland's data only
ire = dataset_ire.to_numpy().reshape(x.shape[0],)

# plot graph with europe and irelands data
fig, ax = plt.subplots(figsize=(15, 8))
ax.plot(x, y, label='Europe')
ax.set_title('Confidence interval for the percent of UAA under organic farming in Europe obtained from sample data over the years')
ax.set_xlabel('Year')
ax.set_ylabel('Percent of UAA under organic farming')
ax.fill_between(x, ll, ul, color='b', alpha=.1)
ax.plot(x, ire, label='Ireland')
plt.legend(loc=2)

Based on the confidence interval calculated it is possible to say that in 2020, for instance, the percent of UAA under organic farming in Europe was between 6.14% and 11.18%.

This interval is accurate, because according to the <a href="https://agridata.ec.europa.eu/extensions/CountryFactsheets/CountryFactsheets.html?memberstate=Ireland#">Agridata</a>, in 2020, the percent of land under organic farming in Europe was about 9%.

## Hypothesis Tests

In this section, two datasets were analyzed using hypotesis tests in order to compare their populations mean and variance.

### Agriculture Import/Export Trade

FAO provides data on the import/export of crops and livestocks products from several countries with data ranging from 1961 up to 2020.

Finland, Slovakia and Ireland have about the same population according to the Word Population Review website, so it is roughly assumed they are similar countries.

In this study, the data about the import and export of agricultural products from these countries are compared using inferential statistics tests.

#### Data Preparation and Visualization

The steps performed to prepare this dataset were implemented in a separate Jupyter notebook called DataPrepVis under this directory.

Thus, the processed datasets were used directly in this notebook.

In [ ]:
# getting import/export val processed dataset
im_ex_val_data = pd.read_csv(join(DATA_PREP_FOLDER, IMP_EXP_DATASET_VAL), encoding='utf-8')
im_ex_val_data.head()

In [ ]:
# getting import/export quantity processed dataset
im_ex_qtt_data = pd.read_csv(join(DATA_PREP_FOLDER, IMP_EXP_DATASET_QTT), encoding='utf-8')
im_ex_qtt_data.head()

As shown below, Slovakia data covers a period of time between 1993 and 2020. 

Finland and Ireland have data starting from 1961, so this information was taken into account when comparing data between the countries.

In [ ]:
# date range for Slovakia dataset
print(im_ex_qtt_data[im_ex_qtt_data['Area']=='Slovakia'].Year.sort_values().unique())

# date range for Ireland dataset
print(im_ex_qtt_data[im_ex_qtt_data['Area']=='Ireland'].Year.sort_values().unique())

# date range for Finland dataset
print(im_ex_qtt_data[im_ex_qtt_data['Area']=='Finland'].Year.sort_values().unique())

The graph below shows the summarized (average) import/export quantities and values per country by year, respectivelly.

This way, it was possible to visualize the differences between the countries better, which helps to explain the results obtained by the inferential tests.

In [ ]:
Image(filename=join(IMAGES_FOLDER, GRAPH_IMP_EXP_QTT))

In [ ]:
Image(filename=join(IMAGES_FOLDER, GRAPH_IMP_EXP_VAL))

According to the graphs, Ireland's import and export numbers are higher than the other countries, mainly in terms of export values in US$ (Graph 4).

Regarding Slovakia and Finland, they import about the same quantity and values, having slighlty different behaviour with regards to the export numbers, where Slovakia shows more exports than Finland.

#### Analysis

In this section the import/export data from each country are compared using inferential statistics tests in order to validate if their trade are similar.

The summarized import/export data are calculated for all countries. Comparisons with Slovakia should use data from 1993 until 2020 only due to Slovakia's lack of older data.

In [ ]:
# getting the average quantity of tonnes of import/export a year
imp_ex_qtt_avg = im_ex_qtt_data.groupby(by=['Year', 'Element', 'Area'], as_index=False).mean().sort_values(by=['Year'], ignore_index=True)

# getting the average value in US$ of import/export a year
imp_ex_val_avg = im_ex_val_data.groupby(by=['Year', 'Element', 'Area'], as_index=False).mean().sort_values(by=['Year'], ignore_index=True)

##### Import/Export probability distributions - Shapiro Test

First it's necessary to check if the import/export data are normally distributted in order to decide which inferential test to apply.

From the distribution plotted below it was observed the data is not normally distributed. Using Shapiro test to confirm that hypotesis.

In [ ]:
# checking import/export quantity average data are normal
for c in list(imp_ex_qtt_avg.Area.unique()):
    for e in list(imp_ex_qtt_avg.Element.unique()):
        data = imp_ex_qtt_avg[(imp_ex_qtt_avg['Area'] == c)&(imp_ex_qtt_avg['Element'] == e)].Value
        plot_col_graph(data, f'Prob. distribution of {e} in {c}')
        print(f'Is {e} from {c} normally distributed? {isDistNormal(data)}')

##### Import/Export between Slovakia and Finland - Kruskal Test

Since data is not normally distributed, Kruskal test will be applied to check if populations of both Finland and Slovakia are similar or not.

Since Slovakia data is being tested, it was used data 1993 until 2020, so both datasets are aligned.

In [ ]:
# compare Slovakia and Finland import quantity data
data = imp_ex_qtt_avg[(imp_ex_qtt_avg['Element'] == 'Import Quantity')&(imp_ex_qtt_avg['Area'].isin(['Slovakia', 'Finland']))&(imp_ex_qtt_avg['Year'] >= 1993)]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Slovakia and Finland import value data
data = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Import Value')&(imp_ex_val_avg['Area'].isin(['Slovakia', 'Finland']))&(imp_ex_val_avg['Year'] >= 1993)]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Slovakia and Finland export quantity data
data = imp_ex_qtt_avg[(imp_ex_qtt_avg['Element'] == 'Export Quantity')&(imp_ex_qtt_avg['Area'].isin(['Slovakia', 'Finland']))&(imp_ex_qtt_avg['Year'] >= 1993)]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Slovakia and Finland export value data
data = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Export Value')&(imp_ex_val_avg['Area'].isin(['Slovakia', 'Finland']))&(imp_ex_val_avg['Year'] >= 1993)]
kruskal(data=data, dv='Value', between='Area')

Based on the results from the Kruskal test and assuming 5% significance level, it was possible to conclude that, Finland and Slovakia are different regarding their export quantity. However, both countries have similar average import value and quantity as well as export value of crops and livestock products.

This result can be verified by checking the graphs in the previous section, where it was observed that the curves for these countries are very close to each other.

##### Import/Export between Ireland and Finland - Kruskal Test

The same test was applied to compare Ireland and Finland.

In [ ]:
# compare Ireland and Finland import quantity data
data = imp_ex_qtt_avg[(imp_ex_qtt_avg['Element'] == 'Import Quantity')&(imp_ex_qtt_avg['Area'].isin(['Ireland', 'Finland']))]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Ireland and Finland import value data
data = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Import Value')&(imp_ex_val_avg['Area'].isin(['Ireland', 'Finland']))]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Ireland and Finland export quantity data
data = imp_ex_qtt_avg[(imp_ex_qtt_avg['Element'] == 'Export Quantity')&(imp_ex_qtt_avg['Area'].isin(['Ireland', 'Finland']))]
kruskal(data=data, dv='Value', between='Area')

In [ ]:
# compare Ireland and Finland export value data
data = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Export Value')&(imp_ex_val_avg['Area'].isin(['Ireland', 'Finland']))]
kruskal(data=data, dv='Value', between='Area')

Based on the results from the Kruskal test and assuming 5% significance level, it was possible to conclude that Ireland and Finland are similar regarding their import values. 

However, there is evidence to say these two countries differ on their import quantity and export quantity and values average.

This result can be verified by checking the graphs in the previous section, where it was observed that these countries curves are distant from each other in most of the cases, expect for their import values.

##### Import/Export between Ireland and Finland - Mann-Whitneyu Test

Using another non-parametric test to confirm results obtained from Kruskal. 

Validating the results when comparing Ireland and Finland trade data.

In [ ]:
# getting only the necessary import data to apply the inferential test
im_value_ire = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Import Value')&(imp_ex_val_avg['Area'] == 'Ireland')]
im_value_fin = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Import Value')&(imp_ex_val_avg['Area'] == 'Finland')]
stats.mannwhitneyu(im_value_ire['Value'], im_value_fin['Value'])

In [ ]:
# getting only the necessary export data to apply the inferential test
im_value_ire = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Export Value')&(imp_ex_val_avg['Area'] == 'Ireland')]
ex_value_ire = imp_ex_val_avg[(imp_ex_val_avg['Element'] == 'Export Value')&(imp_ex_val_avg['Area'] == 'Finland')]
stats.mannwhitneyu(im_value_ire['Value'], ex_value_ire['Value'])

The above results confirm that, with 5% of significance level, Ireland and Finland are similar in the import value only, whereas in the other aspects there is enough evidence to confirm they have different trading average.

#### Results

According to the tests above, Finland and Slovakia are similar countries with regards to import and export of agricultural products. They only differ in the quantity of goods that are exported from each country.

Ireland, on the other hand exports and import much more than the other two countries, which was confirmed by the inferential tests that showed the average of import/export between Ireland and Finland are substantially different.

### Food price inflation

FAO provides data on the food price inflation from several countries with data from 2001 up to 2022.

Finland, Slovakia and Ireland have about the same population according to the Word Population Review website, so it is roughly assumed they are similar countries.

In this study, the data about the food price inflation from these countries are compared using inferential statistics tests.

#### Data Preparation and Visualization

The steps performed to prepare this dataset were implemented in a separate Jupyter notebook called DataPrepVis under this directory.

Thus, the processed datasets were used directly in this notebook.

In [ ]:
# getting the food price inflation processed dataset
dataset = pd.read_csv(join(DATA_PREP_FOLDER, FOOD_INF_DATASET), encoding='utf-8')
dataset.head()

The graph below shows the summarized food price inflation per country by year. 

This way, it is possible to visualize the differences between the countries over the years.

In [ ]:
Image(filename=join(IMAGES_FOLDER, GRAPH_FOOD_PRICE_INF))

According to the graph, Slovakia presents the highest food inflation among the three countries, whereas Ireland has the lowest numbers.

#### Analysis

In this section the food price inflation data from each country are compared using inferential statistics tests in order to validate if their average inflation are similar.

The summarized inflation dataset was obtaind from the processed datasets folder.

In [ ]:
# getting the summarized food price inflation processed dataset
food_inf_avg = pd.read_csv(join(DATA_PREP_FOLDER, FOOD_INF_DATASET_AVG), encoding='utf-8')
food_inf_avg.head()

##### Food price inflation probability distributions - Shapiro Test

First it's necessary to check if the summarized inflation data are normally distributted in order to decide which inferential test to apply.

From the distribution plotted below it was observed the data is normally distributed for Finland and Ireland only. Using Shapiro test to confirm that hypotesis.

In [ ]:
# checking food price inflation data are normal
for c in list(food_inf_avg.Area.unique()):
    data = food_inf_avg[food_inf_avg['Area'] == c].Value
    plotColGraph(data, f'Prob. distribution of food price inflation in {c}')
    print(f'Is food price inflation from {c} normally distributed? {is_dist_normal(data)}')

##### Food price inflation between Ireland and Finland - Wilcoxon Test

Using non-parametric test to compare the aggregated food price inflation between Ireland and Finland.

In [ ]:
# use wilcoxon stats test to compare inflation between Ireland and Finland
wilcoxon(food_inf_avg[food_inf_avg['Area'] == 'Ireland'].Value, food_inf_avg[food_inf_avg['Area'] == 'Finland'].Value)

The result above shows that, with 5% of significance level, the average food price inflation are not similar between Ireland and Finland.

##### Food price inflation between Slovakia and Finland - Wilcoxon Test

Using non-parametric test to compare the aggregated food price inflation between Slovakia and Finland.

In [ ]:
# use wilcoxon stats test to compare inflation between Finland and Slovakia
wilcoxon(food_inf_avg[food_inf_avg['Area'] == 'Slovakia'].Value, food_inf_avg[food_inf_avg['Area'] == 'Finland'].Value)

The result above shows that, with 5% of significance level, the average food price inflation are similar between Slovakia and Finland.

##### Food price inflation between the countries - Levene Test

Using Levene test to check if the food price inflation data between Ireland and the other countries have the same variance.

In [ ]:
# comparing data between Ireland and Slovakia
levene(dataset[dataset['Area'] == 'Ireland'].Value, dataset[dataset['Area'] == 'Slovakia'].Value, center='mean')

In [ ]:
# comparing data between Ireland and Finland
levene(dataset[dataset['Area'] == 'Ireland'].Value, dataset[dataset['Area'] == 'Finland'].Value, center='mean')

In [ ]:
# comparing data between Finland and Slovakia
levene(dataset[dataset['Area'] == 'Finland'].Value, dataset[dataset['Area'] == 'Slovakia'].Value, center='mean')

The above results suggest that, with 5% of significance level, there is no significant difference in the variance of the average food price inflation between Ireland and Finland and Finland and Slovakia. 

However, this result also suggests that, with 5% of significance level, Ireland and Slovakia food price inflation variance are different. 

These results confirm the first analysis of the graph in the Data Preparation and Visualization step.

##### Food price inflation between Finland and Ireland - Anova One Way Test

Using parametric test Anova with Ireland and Finland summarized food price inflation dataset as they met the normality requirement as per Shapiro test.

In [ ]:
# using anova one way to compare Ireland and Finland
model = ols('Value~Area', data=food_inf_avg[food_inf_avg['Area'].isin(['Ireland', 'Finland'])]).fit()
aov = sm.stats.anova_lm(model, type=2)
print(aov)

Based on the above result, with 5% of significance level, there is evidence to say that Ireland and Finland have similar food price inflation.

#### Results

By using different parametric and non-parametric inferential statistics tests it was possible to confirm and validate the food price inflation differences between the countries.

Ireland and Finland have similar inflation, while Ireland and Slovakia have significant differences.